In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import plotly.express as px
from scipy.stats.mstats import trimmed_var
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/unsupervised-learning-on-country-data/Country-data.csv
/kaggle/input/unsupervised-learning-on-country-data/data-dictionary.csv


# Import CSV file

In [2]:
df = pd.read_csv("/kaggle/input/unsupervised-learning-on-country-data/Country-data.csv")
df.shape

(167, 10)

In [3]:
df.head()

,country,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,Afghanistan,90.2,10.0,7.58,44.9,1610,9.44,56.2,5.82,553
1,Albania,16.6,28.0,6.55,48.6,9930,4.49,76.3,1.65,4090
2,Algeria,27.3,38.4,4.17,31.4,12900,16.10,76.5,2.89,4460
3,Angola,119.0,62.3,2.85,42.9,5900,22.40,60.1,6.16,3530
4,Antigua and Barbuda,10.3,45.5,6.03,58.9,19100,1.44,76.8,2.13,12200


In [4]:
df_var = df.var().sort_values()
df_var

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


total_fer     2.291734e+00
health        7.545116e+00
life_expec    7.908851e+01
inflation     1.117398e+02
imports       5.861042e+02
exports       7.514183e+02
child_mort    1.626423e+03
gdpp          3.359414e+08
income        3.716439e+08
dtype: float64

## EDA

In [5]:
fig = px.bar(df_var,orientation="h",title="Varience Distribution")
fig.show()

In [6]:
fig = px.box(
    x=df["income"],
    title="Income distribution"
)
fig.show()

In [7]:
fig = px.box(
    x=df["gdpp"],
    title="GDP distribution"
)
fig.show()

### Select numeric datatypes for clustering

In [8]:
df_num = df.select_dtypes(np.number)
df_num.head()

,child_mort,exports,health,imports,income,inflation,life_expec,total_fer,gdpp
0,90.2,10.0,7.58,44.9,1610,9.44,56.2,5.82,553
1,16.6,28.0,6.55,48.6,9930,4.49,76.3,1.65,4090
2,27.3,38.4,4.17,31.4,12900,16.10,76.5,2.89,4460
3,119.0,62.3,2.85,42.9,5900,22.40,60.1,6.16,3530
4,10.3,45.5,6.03,58.9,19100,1.44,76.8,2.13,12200


In [9]:
fig = px.box(df_num)
fig.show()

The boxplot shows difference in range of values hence we'll use standard scalar. But first we'll find the trimmed varience and make a bar chart of trimmed variance.

In [10]:
df_trim = df_num.apply(trimmed_var,limits=(0.1,0.2)).sort_values().to_frame()
df_trim

,0
total_fer,6.376198e-01
health,2.250650e+00
inflation,1.111651e+01
life_expec,3.495236e+01
imports,1.253018e+02
exports,1.267170e+02
child_mort,3.373467e+02
gdpp,2.591492e+07
income,5.698423e+07


In [11]:
fig = px.bar(df_trim,orientation="h",title="Trimmed Varience Distribution")
fig.show()

We'll take first five columns with highest trimmed varience and do the vertical split to define our training set X. 

In [12]:
top_five_high_var = df_trim.tail().index.to_list()
top_five_high_var

['imports', 'exports', 'child_mort', 'gdpp', 'income']

In [13]:
#Vertical Split
X =df[top_five_high_var]
X.head()

,imports,exports,child_mort,gdpp,income
0,44.9,10.0,90.2,553,1610
1,48.6,28.0,16.6,4090,9930
2,31.4,38.4,27.3,4460,12900
3,42.9,62.3,119.0,3530,5900
4,58.9,45.5,10.3,12200,19100


In [14]:
#To study how standard scaler works...can be omitted
# Instantiate transformer
ss = StandardScaler()

# Transform `X`
X_scaled_data = ss.fit_transform(X)
type(X_scaled_data)

# Put `X_scaled_data` into DataFrame
X_scaled = pd.DataFrame(X_scaled_data, columns=X.columns)

print("X_scaled shape:", X_scaled.shape)
X_scaled.tail()

X_scaled shape: (167, 5)


,imports,exports,child_mort,gdpp,income
162,0.240700,0.200917,-0.225578,-0.546913,-0.738527
163,-1.213499,-0.461363,-0.526514,0.029323,-0.033542
164,1.380030,1.130305,-0.372315,-0.637754,-0.658404
165,-0.517472,-0.406478,0.448417,-0.637754,-0.658924
166,-0.662477,-0.150348,1.114951,-0.629546,-0.721358


In [15]:
#To check that X is scaled
X_summary = X_scaled.aggregate(["mean","std"]).astype(int)
X_summary

,imports,exports,child_mort,gdpp,income
mean,0,0,0,0,0
std,1,1,1,1,1


In [16]:
# #to find optimal number of clusters
# n_clusters = range(2,13)
# inertia_errors = []
# silhouette_scores = []

# # Add `for` loop to train model and calculate inertia, silhouette score.
# for k in n_clusters:
#     #Build pipeline
#     model = make_pipeline(
#         StandardScaler(),
#         KMeans(n_clusters=k,random_state=42)
#     )
#     model.fit(X)
#     #Calculate inertia and append to inertia_errors list
#     inertia_errors.append(model.named_steps["kmeans"].inertia_)
    
#     #Calculate silhouette score and append to silhouette_scores list
#     silhouette_scores.append(
#         silhouette_score(X,model.named_steps["kmeans"].labels_))

# print("Inertia:", inertia_errors[:5])
# print()
# print("Silhouette Scores:", silhouette_scores[:5])

## Build model pipeline

In [17]:
final_model = make_pipeline(
    StandardScaler(),
    KMeans(n_clusters=4, random_state=42)
)
final_model.fit(X)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kmeans', KMeans(n_clusters=4, random_state=42))])

In [18]:
labels = final_model.named_steps["kmeans"].labels_
print(labels[:5])

[2 0 0 2 0]


In [19]:
xgb = X.groupby(labels).mean()
# Create side-by-side bar chart of `xgb`
fig = px.bar(
    xgb,
    barmode="group"
)
fig.update_layout(xaxis_title="Cluster",yaxis_title="Value [$]",
                 title_text="Mean by Cluster")
fig.show()

In [20]:
#PCA using standard scaler in a pipeline
X_t1 = make_pipeline(
    StandardScaler(),
    PCA(n_components=2,random_state=42)
)
X_transformed = X_t1.fit_transform(X)

X_pca1 = pd.DataFrame(X_transformed,columns=["PC1","PC2"])

In [21]:
# Creating scatter plot of `PC2` vs `PC1` when using standard scaler in a pipeline
fig = px.scatter(
    data_frame=X_pca1,
    x="PC1",
    y="PC2",
    color=labels.astype(str),#to make the colors distinct for each cluster in the legend
    title="PCA Representation of Clusters"
)
fig.update_layout(xaxis_title="PC1",yaxis_title="PC2")
fig.show()